## Исследование влияния dropout на качество прогнозирования модели LSTM

### Подключение библиотек

In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.keras.optimizers import Adam

import sys
import os
sys.path.append(os.path.abspath('..'))
from helpful_functions import train_val_test_split, \
                              create_sequences, \
                              train_and_evaluate_model

### Подготовка данных

In [11]:
file_path = '../dataset/preprocessed_data.csv'
df = pd.read_csv(file_path)

df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)

target_feature = 'Global_active_power'
test_ratio = 0.10
val_ratio = 0.10

train_data, val_data, test_data = train_val_test_split(df, val_ratio, test_ratio)

scaler = MinMaxScaler(feature_range=(0, 1))
train_scaled = scaler.fit_transform(train_data)
val_scaled = scaler.transform(val_data)
test_scaled = scaler.transform(test_data)

target_idx = df.columns.get_loc(target_feature)

features_count = train_data.shape[1]

### Обучение моделей

In [12]:
lr = 0.01
window_size = 30

X_train, y_train = create_sequences(train_scaled, target_idx, window_size)
X_val, y_val = create_sequences(val_scaled, target_idx, window_size)
X_test, y_test = create_sequences(test_scaled, target_idx, window_size)

dropout_configs = [
    [('none', 0), ('none', 0)], 
    [('after_first', 0.2), ('none', 0)],
    [('after_first', 0.3), ('none', 0)],
    [('after_first', 0.5), ('none', 0)],
    [('none', 0), ('after_second', 0.2)],
    [('none', 0), ('after_second', 0.3)],
    [('none', 0), ('after_second', 0.5)],
    [('after_first', 0.2), ('after_second', 0.2)],
    [('after_first', 0.3), ('after_second', 0.3)],
    [('after_first', 0.5), ('after_second', 0.5)],
]

results = {}

for config in dropout_configs:
    model_name = "LSTM"
    model = Sequential()
    model.add(Input(shape=(window_size, features_count)))

    model.add(LSTM(100, activation='tanh', return_sequences=True))
    if config[0][0] == 'after_first':
        model.add(Dropout(config[0][1]))
        model_name += f"_Dropout1_{config[0][1]}"

    model.add(LSTM(50, activation='tanh'))
    if config[1][0] == 'after_second':
        model.add(Dropout(config[1][1]))
        model_name += f"_Dropout2_{config[1][1]}"

    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
    
    print(f"\nОбучение модели: {model_name}")
    result = train_and_evaluate_model(
        model, model_name, X_train, y_train, X_val, y_val, X_test, y_test,
        lr, window_size, scaler, target_feature, df.columns
    )

    results[model_name] = result


Обучение модели: LSTM
LSTM | val_loss: 0.0003 | MSE_val: 0.0421

Обучение модели: LSTM_Dropout1_0.2
LSTM_Dropout1_0.2 | val_loss: 0.0003 | MSE_val: 0.0383

Обучение модели: LSTM_Dropout1_0.3
LSTM_Dropout1_0.3 | val_loss: 0.0003 | MSE_val: 0.0394

Обучение модели: LSTM_Dropout1_0.5
LSTM_Dropout1_0.5 | val_loss: 0.0003 | MSE_val: 0.0403

Обучение модели: LSTM_Dropout2_0.2
LSTM_Dropout2_0.2 | val_loss: 0.0003 | MSE_val: 0.0403

Обучение модели: LSTM_Dropout2_0.3
LSTM_Dropout2_0.3 | val_loss: 0.0003 | MSE_val: 0.0397

Обучение модели: LSTM_Dropout2_0.5
LSTM_Dropout2_0.5 | val_loss: 0.0003 | MSE_val: 0.0421

Обучение модели: LSTM_Dropout1_0.2_Dropout2_0.2
LSTM_Dropout1_0.2_Dropout2_0.2 | val_loss: 0.0003 | MSE_val: 0.0417

Обучение модели: LSTM_Dropout1_0.3_Dropout2_0.3
LSTM_Dropout1_0.3_Dropout2_0.3 | val_loss: 0.0004 | MSE_val: 0.0444

Обучение модели: LSTM_Dropout1_0.5_Dropout2_0.5
LSTM_Dropout1_0.5_Dropout2_0.5 | val_loss: 0.0013 | MSE_val: 0.1532


### Вывод информации всех результатов обучения модели

In [13]:
results_df = pd.DataFrame(results.values())
print(results_df.sort_values(by=['mse_val_dn']))

                       model_type  learning_rate  window_size  train_loss  \
1               LSTM_Dropout1_0.2           0.01           30    0.000459   
2               LSTM_Dropout1_0.3           0.01           30    0.000467   
5               LSTM_Dropout2_0.3           0.01           30    0.000529   
3               LSTM_Dropout1_0.5           0.01           30    0.000494   
4               LSTM_Dropout2_0.2           0.01           30    0.000492   
7  LSTM_Dropout1_0.2_Dropout2_0.2           0.01           30    0.000525   
0                            LSTM           0.01           30    0.000426   
6               LSTM_Dropout2_0.5           0.01           30    0.000651   
8  LSTM_Dropout1_0.3_Dropout2_0.3           0.01           30    0.000602   
9  LSTM_Dropout1_0.5_Dropout2_0.5           0.01           30    0.001907   

   val_loss  mse_val_dn  mse_test_dn  
1  0.000314    0.038322     0.032350  
2  0.000323    0.039385     0.033443  
5  0.000325    0.039697     0.03321

### Вывод информации лучшей модели

In [14]:
lstm_results = results_df[results_df['model_type'].str.startswith('LSTM_')]
best_lstm = lstm_results.nsmallest(1, 'mse_val_dn').iloc[0]

print("\nЛучшая LSTM (по валидации):")
print(best_lstm[['model_type', 'learning_rate', 'window_size', 'mse_val_dn', 'mse_test_dn']])


Лучшая LSTM (по валидации):
model_type       LSTM_Dropout1_0.2
learning_rate                 0.01
window_size                     30
mse_val_dn                0.038322
mse_test_dn                0.03235
Name: 1, dtype: object
